In [1]:
%matplotlib inline

datapath = "../../data/dl1/obj_neural_code/"
flatdatapath = "../../data/dl1/obj_neural_code/all_flat/"
repath   = "../../data/dl1/obj_neural_code/rep/finetune/"
N = 1440

In [2]:
from time import time
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import scipy.spatial.distance
import utils; reload(utils)
from utils import plots
import vgg16; reload(vgg16)
from vgg16 import Vgg16
vgg = Vgg16()

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [29]:
from keras.models import Model

idx = [0,5,10,17,24,31,33,35,37]
# lambda_1
# maxpooling2d_1
# maxpooling2d_2
# maxpooling2d_3
# maxpooling2d_4
# maxpooling2d_5
# dense_1
# dense_2
# dense_3

def get_models(idx,base):
    layers  = []
    for i in range(len(idx)):
        print(base.layers[idx[i]].name)
        layers.append( base.layers[idx[i]] )

    models = []
    outshapes = []
    for layer in layers:
        models.append( Model(input=base.input,
                             output=base.get_layer(layer.name).output) )
        outshapes.append(list( layer.output_shape ))
        
        
    print('Done.')
    return layers,models,outshapes

In [5]:
batch_size = 60
nbatches = int(N/batch_size)
batches = vgg.get_batches(datapath+'train', batch_size=batch_size)
val_batches = vgg.get_batches(datapath+'valid', batch_size=batch_size)

Found 1120 images belonging to 40 classes.
Found 160 images belonging to 40 classes.


In [6]:
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=10)

Epoch 1/10
1120/1120 [==============================] - 31s - loss: 3.8764 - acc: 0.1929 - val_loss: 1.1506 - val_acc: 0.6250
Epoch 2/10
1120/1120 [==============================] - 31s - loss: 1.3285 - acc: 0.6071 - val_loss: 0.6097 - val_acc: 0.7937
Epoch 3/10
1120/1120 [==============================] - 31s - loss: 0.9261 - acc: 0.7170 - val_loss: 0.5714 - val_acc: 0.7937
Epoch 4/10
1120/1120 [==============================] - 30s - loss: 0.7639 - acc: 0.7696 - val_loss: 0.4275 - val_acc: 0.8563
Epoch 5/10
1120/1120 [==============================] - 31s - loss: 0.5970 - acc: 0.8098 - val_loss: 0.3872 - val_acc: 0.9125
Epoch 6/10
1120/1120 [==============================] - 31s - loss: 0.4678 - acc: 0.8438 - val_loss: 0.3431 - val_acc: 0.8937
Epoch 7/10
1120/1120 [==============================] - 31s - loss: 0.4001 - acc: 0.8705 - val_loss: 0.4243 - val_acc: 0.8375
Epoch 8/10
1120/1120 [==============================] - 31s - loss: 0.3661 - acc: 0.8884 - val_loss: 0.3078 - val_acc:

In [30]:
base = vgg.model
layers,models,outshapes = get_models(idx,base)

lambda_1
maxpooling2d_1
maxpooling2d_2
maxpooling2d_3
maxpooling2d_4
maxpooling2d_5
dense_1
dense_2
dense_4
Done.


In [33]:
for i in range(len(outshapes)):
    outshapes[i][0] = N
    print(outshapes[i])

[1440, 3, 224, 224]
[1440, 64, 112, 112]
[1440, 128, 56, 56]
[1440, 256, 28, 28]
[1440, 512, 14, 14]
[1440, 512, 7, 7]
[1440, 4096]
[1440, 4096]
[1440, 40]


In [34]:
D = []
D_sq = []
for layer,model in zip(layers,models):
    print('Processing model : ' + layer.name)
    R = []
    batches = vgg.get_batches(flatdatapath, batch_size=batch_size, shuffle=False)
    ti = time()
    for n in range(nbatches):
        imgs,_ = batches.next()       
        R.append(model.predict(imgs))
    R = np.asarray(R, dtype=float) 
    outshape = list(layer.output_shape)
    outshape[0] = N
    R.shape = tuple(outshape)
    print(R.shape)
    np.save(repath +  layer.name, R)
        
    # compute distances
    R.shape = R.shape[0],-1
    d = scipy.spatial.distance.pdist(R, 'euclidean')
    d_sq = scipy.spatial.distance.squareform(d, force='no', checks=True)
    D.append(d)
    D_sq.append(d_sq)
    te = time() - ti
    print(te) 
    
np.save(repath + 'D', D)
np.save(repath + 'D_sq', D_sq)
print('Done.')

Processing model : lambda_1
Found 1440 images belonging to 1 classes.
(1440, 3, 224, 224)
228.873123169
Processing model : maxpooling2d_1
Found 1440 images belonging to 1 classes.
(1440, 64, 112, 112)
1273.38632202
Processing model : maxpooling2d_2
Found 1440 images belonging to 1 classes.
(1440, 128, 56, 56)
675.266762018
Processing model : maxpooling2d_3
Found 1440 images belonging to 1 classes.
(1440, 256, 28, 28)
358.082916975
Processing model : maxpooling2d_4
Found 1440 images belonging to 1 classes.
(1440, 512, 14, 14)
207.632259846
Processing model : maxpooling2d_5
Found 1440 images belonging to 1 classes.
(1440, 512, 7, 7)
90.8472390175
Processing model : dense_1
Found 1440 images belonging to 1 classes.
(1440, 4096)
58.68516922
Processing model : dense_2
Found 1440 images belonging to 1 classes.
(1440, 4096)
59.9352638721
Processing model : dense_4
Found 1440 images belonging to 1 classes.
(1440, 40)
54.7917461395
Done.


In [45]:
len(D_sq)

9

In [46]:
D_sq[7]

array([[  0.    ,  67.4861,  73.3515, ...,  67.5389,  99.9265,  54.425 ],
       [ 67.4861,   0.    ,  71.9777, ...,  67.278 ,  92.3884,  54.0586],
       [ 73.3515,  71.9777,   0.    , ...,  62.1377,  80.3235,  57.9102],
       ..., 
       [ 67.5389,  67.278 ,  62.1377, ...,   0.    ,  76.0377,  49.7512],
       [ 99.9265,  92.3884,  80.3235, ...,  76.0377,   0.    ,  83.7193],
       [ 54.425 ,  54.0586,  57.9102, ...,  49.7512,  83.7193,   0.    ]])